To be up-to-date on the most current version of this code. Check out my GitHub repository: https://github.com/Neatherblok/SnowDetection

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

## Data Preparation

In [ ]:
from Data_Preparation.Preparation import CustomDataLoader

In [ ]:
# training data properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
BATCH_SIZE = 32

In [ ]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="./data", batch_size=BATCH_SIZE, dataset_type="train", mean=MEAN, std=STD).data_loader
test_data_loader = CustomDataLoader(data_path="./data", batch_size=BATCH_SIZE, dataset_type="test", mean=MEAN, std=STD).data_loader

image_datasets = {'train':train_data_loader.dataset, 'val':test_data_loader.dataset}
dataloaders = {'train':train_data_loader, 'val':test_data_loader}

## Initializing VGG19 and ResNet50 Finetuning

In [ ]:
# Load pre-trained models
vgg19 = models.vgg19(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

In [ ]:
# Freeze parameters so we don't backprop through them
for param in vgg19.parameters():
    param.requires_grad = False
for param in resnet50.parameters():
    param.requires_grad = False

In [ ]:
# Replace the classifier with a new one
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
resnet50.fc = nn.Linear(2048, num_classes)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_vgg19 = optim.Adam(vgg19.parameters())
optimizer_resnet50 = optim.Adam(resnet50.parameters())

## Training VGG19 and ResNet50

In [ ]:
# Train the models
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
            resnet50.train()
        else:
            vgg19.eval()
            resnet50.eval()

        running_loss = 0.0
        corrects = 0

        for inputs, labels in dataloaders[phase]:
            optimizer_vgg19.zero_grad()
            optimizer_resnet50.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                outputs_resnet50 = resnet50(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)
                _, preds_resnet50 = torch.max(outputs_resnet50, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)
                loss_resnet50 = criterion(outputs_resnet50, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    loss_resnet50.backward()
                    optimizer_vgg19.step()
                    optimizer_resnet50.step()

            running_loss += loss_vgg19.item() * inputs.size(0)
            running_loss += loss_resnet50.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)
            corrects += torch.sum(preds_resnet50 == labels.data)

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = corrects.double() / len(image_datasets[phase])

        print('{} Loss: {:.4f} | Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

## Ensemble Models

In [ ]:
# Ensemble predictions
def ensemble_predict(model1, model2, dataloader):
    predictions = []
    for inputs, labels in dataloader:
        outputs1 = model1(inputs)
        outputs2 = model2(inputs)
        outputs = (outputs1 + outputs2) / 2  # Simple averaging
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.tolist())
    return {'pred':predictions, 'label':labels}

ensemble_predictions = ensemble_predict(vgg19, resnet50, dataloaders['val'])

In [ ]:
print(ensemble_predictions)